In [1]:
%pip install git+https://github.com/haccht/scrapli_ipython
%reload_ext scrapli_ipython

  Cloning https://github.com/haccht/scrapli_ipython to /tmp/pip-req-build-kz6c_gu8
  Running command git clone --filter=blob:none --quiet https://github.com/haccht/scrapli_ipython /tmp/pip-req-build-kz6c_gu8
  Resolved https://github.com/haccht/scrapli_ipython to commit 68f76807f089a0b465b1b94619994b3c03a65e41
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


You can establish a connection to a device with `%scrapli` line magic.

In [2]:
%platform cisco_iosxr
%telnet 192.168.121.105

Username: cisco
Password: ········





IMPORTANT:  READ CAREFULLY
Welcome to the Demo Version of Cisco IOS XRv (the "Software").
The Software is subject to and governed by the terms and conditions
of the End User License Agreement and the Supplemental End User
License Agreement accompanying the product, made available at the
time of your order, or posted on the Cisco website at
www.cisco.com/go/terms (collectively, the "Agreement").
As set forth more fully in the Agreement, use of the Software is
strictly limited to internal use in a non-production environment
solely for demonstration and evaluation purposes.  Downloading,
installing, or using the Software constitutes acceptance of the
Agreement, and you are binding yourself and the business entity
that you represent to the Agreement.  If you do not agree to all
of the terms of the Agreement, then Cisco is unwilling to license
the Software to you and (a) you may not download, install or use the
Software, and (b) you may return the Software as more fully set forth
in the

`%%cmd` cell magic allows you to execute all commands in the cell.

In [3]:
%%cmd
show interface brief
show ipv4 interface brief


RP/0/0/CPU0:iosxrv#show interface brief
Fri Dec  9 15:58:56.261 UTC

               Intf       Intf        LineP              Encap  MTU        BW
               Name       State       State               Type (byte)    (Kbps)
--------------------------------------------------------------------------------
                Nu0          up          up               Null  1500          0
       Mg0/0/CPU0/0          up          up               ARPA  1514          0
          Gi0/0/0/0          up          up               ARPA  1514    1000000
          Gi0/0/0/1          up          up               ARPA  1514    1000000
          Gi0/0/0/2          up          up               ARPA  1514    1000000

RP/0/0/CPU0:iosxrv#show ipv4 interface brief
Fri Dec  9 15:58:56.421 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
MgmtEth0/0/CPU0/0              192.168.121.105 Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Up     

`%%cmd` cell magic accepts a jinja2 template and execute the rendered commands.

In [4]:
%%cmd resp
{% for i in range(3) %}
show interface Gi0/0/0/{{i}}
{% endfor %}


RP/0/0/CPU0:iosxrv#show interface Gi0/0/0/0
Fri Dec  9 15:59:01.000 UTC
GigabitEthernet0/0/0/0 is up, line protocol is up 
  Interface state transitions: 7
  Hardware is GigabitEthernet, address is 5254.00dd.7f8c (bia 5254.00dd.7f8c)
  Internet address is Unknown
  MTU 1514 bytes, BW 1000000 Kbit (Max: 1000000 Kbit)
     reliability 255/255, txload 0/255, rxload 0/255
  Encapsulation ARPA,
  Full-duplex, 1000Mb/s, unknown, link type is force-up
  output flow control is off, input flow control is off
  Carrier delay (up) is 10 msec
  loopback not set,
  Last link flapped 3w6d
  Last input 00:00:00, output never
  Last clearing of "show interface" counters never
  5 minute input rate 0 bits/sec, 1 packets/sec
  5 minute output rate 0 bits/sec, 0 packets/sec
     1237036 packets input, 74222160 bytes, 0 total input drops
     0 drops for unrecognized upper-level protocol
     Received 0 broadcast packets, 1237036 multicast packets
              0 runts, 0 giants, 0 throttles, 0 parity
  

`%%cmd` cell magic can also capture command results as a scrapli MultiReponse, so that you can manupulate them.

In [5]:
resp[0].textfsm_parse_output()

[{'interface': 'GigabitEthernet0/0/0/0',
  'link_status': 'up',
  'admin_state': 'up',
  'hardware_type': 'GigabitEthernet',
  'address': '5254.00dd.7f8c',
  'bia': '5254.00dd.7f8c',
  'description': '',
  'ip_address': 'Unknown',
  'mtu': '1514',
  'duplex': 'Full',
  'speed': '1000Mb/s',
  'bandwidth': '1000000 Kbit',
  'encapsulation': 'ARPA'}]

`%%configure` cell magic loads cell contents into the configuration.

In [6]:
%%configure
{% for i in range(3) %}
no interface Gi0/0/0/{{i}} shutdown
{% endfor %}

show configuration
commit


RP/0/0/CPU0:iosxrv#
RP/0/0/CPU0:iosxrv#configure terminal
Fri Dec  9 15:59:35.548 UTC
RP/0/0/CPU0:iosxrv(config)#
RP/0/0/CPU0:iosxrv(config)#no interface Gi0/0/0/0 shutdown
RP/0/0/CPU0:iosxrv(config)#no interface Gi0/0/0/1 shutdown
RP/0/0/CPU0:iosxrv(config)#no interface Gi0/0/0/2 shutdown
RP/0/0/CPU0:iosxrv(config)#show configuration
Fri Dec  9 15:59:36.428 UTC
Building configuration...
!! IOS XR Configuration 5.3.3
interface GigabitEthernet0/0/0/0
 no shutdown
!
interface GigabitEthernet0/0/0/1
 no shutdown
!
interface GigabitEthernet0/0/0/2
 no shutdown
!
end

RP/0/0/CPU0:iosxrv(config)#commit
Fri Dec  9 15:59:36.848 UTC
RP/0/0/CPU0:iosxrv(config)#

Let's validate whether all ports were successfully opened.

In [7]:
%%cmd resp
{% for i in range(3) %}
show interface Gi0/0/0/{{i}}
{% endfor %}


RP/0/0/CPU0:iosxrv(config)#
RP/0/0/CPU0:iosxrv(config)#end
RP/0/0/CPU0:iosxrv#
RP/0/0/CPU0:iosxrv#show interface Gi0/0/0/0
Fri Dec  9 15:59:59.186 UTC
GigabitEthernet0/0/0/0 is up, line protocol is up 
  Interface state transitions: 7
  Hardware is GigabitEthernet, address is 5254.00dd.7f8c (bia 5254.00dd.7f8c)
  Internet address is Unknown
  MTU 1514 bytes, BW 1000000 Kbit (Max: 1000000 Kbit)
     reliability 255/255, txload 0/255, rxload 0/255
  Encapsulation ARPA,
  Full-duplex, 1000Mb/s, unknown, link type is force-up
  output flow control is off, input flow control is off
  Carrier delay (up) is 10 msec
  loopback not set,
  Last link flapped 3w6d
  Last input 00:00:00, output never
  Last clearing of "show interface" counters never
  5 minute input rate 0 bits/sec, 0 packets/sec
  5 minute output rate 0 bits/sec, 0 packets/sec
     1237065 packets input, 74223900 bytes, 0 total input drops
     0 drops for unrecognized upper-level protocol
     Received 0 broadcast packets, 1237

In [8]:
for i in range(3):
    assert 'administratively down' not in resp[i].result, f"Gi0/0/0/{i} should not be shutdown"

In [9]:
%close


RP/0/0/CPU0:iosxrv#